In [ ]:
# Colabis kasutamiseks
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/thesis/')

!pip install estnltk==1.7.4
!pip install evaluate
!pip install seqeval
!pip install nervaluate

In [ ]:
from modules.data_processing import DatasetProcessor
from modules.bert_data_processing import BERTDataProcessor
from modules.bert_evaluator import BERTEvaluator
from modules.bert_trainer import BERTTrainer
import json
ALL_TAGS = DatasetProcessor.ALL_TAGS
TAG2IDX = DatasetProcessor.TAG2IDX
IDX2TAG = DatasetProcessor.IDX2TAG

In [ ]:
def train_model(model_name, dataset_name, epochs=3, batch_size=16):
    if dataset_name.lower() == 'combined':
      ewt_processor = DatasetProcessor('ewt', from_json=True)
      edt_processor = DatasetProcessor('edt', from_json=True)
      ewt_dataset = ewt_processor.dataset
      edt_dataset = edt_processor.dataset
      dataset = DatasetProcessor.combine_datasetdicts(ewt_dataset, edt_dataset)
    elif dataset_name.lower() in ['ewt', 'edt']:
      processor = DatasetProcessor(dataset_name, from_json=True)
      dataset = processor.dataset

    print(f'{dataset_name.upper()} andmestik laetud')
    bert_processor = BERTDataProcessor(model_name)
    evaluator = BERTEvaluator(all_tags=ALL_TAGS)

    tokenized_dataset = bert_processor.tokenize_dataset(dataset)

    trainer = BERTTrainer(model_name=model_name, idx2tag=IDX2TAG, tag2idx=TAG2IDX, evaluator=evaluator)

    model, model_trainer = trainer.finetune_model(processor=bert_processor, tokenized_dataset=tokenized_dataset, epochs=epochs, batch_size=batch_size, early_stop_patience=3, output_dir=f'./results/{model_name.split("/")[1]}/{dataset_name}')

    results = evaluator.evaluate_and_print(tokenized_dataset['test'], model_trainer)
    evaluator.evaluation_to_json(nervaluate_strict_overall=results[1]['strict'], nervaluate_by_tag=results[2], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on=dataset_name, epochs=epochs)
    

    if dataset_name.lower() == 'combined':
      tokenized_ewt = bert_processor.tokenize_dataset(ewt_dataset['test'])
      tokenized_edt = bert_processor.tokenize_dataset(edt_dataset['test'])

      print(f'Kombineeritud andmestikul treenitud {model_name} EWT testandmestikul')
      ewt_results = evaluator.evaluate_and_print(tokenized_ewt, model_trainer)
      evaluator.evaluation_to_json(nervaluate_strict_overall=ewt_results[1]['strict'], nervaluate_by_tag=ewt_results[2], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on="EWT", epochs=epochs)
      
      print(f'Kombineeritud andmestikul treenitud {model_name} EDT testandmestikul')
      edt_results = evaluator.evaluate_and_print(tokenized_edt, model_trainer)
      evaluator.evaluation_to_json(nervaluate_strict_overall=edt_results[1]['strict'], nervaluate_by_tag=edt_results[2], model_name=model_name.split('/')[1], trained_on=dataset_name, evaluated_on="EDT", epochs=epochs)

      return model, model_trainer, (results, ewt_results, edt_results)

    return model, model_trainer, results

def train_all():
  dataset_names = ['ewt', 'edt', 'combined']

  model_names = ["tartuNLP/EstRoBERTa", "tartuNLP/EstBERT"]
  results_dict = {}

  for model_name in model_names:
    model_results = {}
    for dataset_name in dataset_names:
      model, model_trainer, results = train_model(model_name, dataset_name, epochs=3)
      model_results[dataset_name] = results
    results_dict[model_name] = model_results

  return results_dict

In [4]:
def main():
    results = train_all()

In [5]:
main()

EWT andmestik laetud


tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


Seqeval tulemused
EVE {'precision': 0.5517241379310345, 'recall': 0.5614035087719298, 'f1': 0.5565217391304348, 'number': 57}
GEP {'precision': 0.2631578947368421, 'recall': 0.7272727272727273, 'f1': 0.3864734299516908, 'number': 55}
LOC {'precision': 0.4864864864864865, 'recall': 0.6545454545454545, 'f1': 0.5581395348837209, 'number': 55}
ORG {'precision': 0.7235294117647059, 'recall': 0.5324675324675324, 'f1': 0.6134663341645885, 'number': 231}
PER {'precision': 0.9608879492600423, 'recall': 0.8301369863013699, 'f1': 0.8907398334149927, 'number': 1095}
PROD {'precision': 0.5, 'recall': 0.34806629834254144, 'f1': 0.41042345276872966, 'number': 362}
overall_precision 0.7663438256658596
overall_recall 0.6824797843665769
overall_f1 0.7219846022241232
overall_accuracy 0.9505855469865753

Nervaluate tulemused
Strict {'correct': 1266, 'incorrect': 260, 'partial': 0, 'missed': 329, 'spurious': 137, 'possible': 1855, 'actual': 1663, 'precision': 0.7612748045700541, 'recall': 0.682479784366576

Seqeval tulemused
EVE {'precision': 0.5614035087719298, 'recall': 0.38095238095238093, 'f1': 0.45390070921985815, 'number': 84}
GEP {'precision': 0.6736596736596736, 'recall': 0.7526041666666666, 'f1': 0.7109471094710947, 'number': 384}
LOC {'precision': 0.78125, 'recall': 0.7369614512471655, 'f1': 0.7584597432905484, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7414030261348006, 'recall': 0.7393689986282579, 'f1': 0.7403846153846154, 'number': 729}
PER {'precision': 0.9294478527607362, 'recall': 0.9114740008594757, 'f1': 0.9203731829030157, 'number': 2327}
PROD {'precision': 0.491679273827534, 'recall': 0.5508474576271186, 'f1': 0.5195843325339727, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7941819772528433
overall_recall 0.7957484111330265
overall_f1 0.7949644225506294
overall_accuracy 0.9759567818835686

Nervaluate tulemused
Strict {'correct': 3631, 'incorrect': 694, 'partial': 

In [7]:
estroberta_edt, estroberta_edt_trainer, estroberta_edt_results = train_model('tartuNLP/EstRoBERTa', 'edt', epochs=6)

EDT andmestik laetud


Map:   0%|          | 0/24601 [00:00<?, ? examples/s]

Map:   0%|          | 0/3122 [00:00<?, ? examples/s]

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstRoBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Eve precision,Eve recall,Eve f1,Eve number,Gep precision,Gep recall,Gep f1,Gep number,Loc precision,Loc recall,Loc f1,Loc number,Muu precision,Muu recall,Muu f1,Muu number,Org precision,Org recall,Org f1,Org number,Per precision,Per recall,Per f1,Per number,Prod precision,Prod recall,Prod f1,Prod number,Unk precision,Unk recall,Unk f1,Unk number
1,0.078200,0.085064,0.725127,0.699046,0.711848,0.976293,0.517857,0.325843,0.400000,89,0.597134,0.730994,0.657318,513,0.790991,0.528916,0.633935,830,0.000000,0.000000,0.000000,9,0.534636,0.765903,0.629707,393,0.870056,0.913405,0.891204,1686,0.470109,0.328273,0.386592,527,0.000000,0.000000,0.000000,40
2,0.047600,0.087196,0.715017,0.717641,0.716327,0.976666,0.500000,0.438202,0.467066,89,0.570461,0.820663,0.673062,513,0.793907,0.533735,0.638329,830,0.000000,0.000000,0.000000,9,0.548043,0.783715,0.645026,393,0.885313,0.897390,0.891311,1686,0.457330,0.396584,0.424797,527,0.000000,0.000000,0.000000,40
3,0.031100,0.091934,0.725700,0.728897,0.727295,0.978049,0.545455,0.471910,0.506024,89,0.568270,0.851852,0.681747,513,0.832402,0.538554,0.653987,830,0.000000,0.000000,0.000000,9,0.545283,0.735369,0.626219,393,0.914442,0.893832,0.904019,1686,0.472426,0.487666,0.479925,527,0.000000,0.000000,0.000000,40
4,0.019400,0.101696,0.741753,0.737216,0.739477,0.978903,0.603175,0.426966,0.500000,89,0.561470,0.863548,0.680492,513,0.787540,0.593976,0.677198,830,0.000000,0.000000,0.000000,9,0.614894,0.735369,0.669757,393,0.909635,0.901542,0.905570,1686,0.519187,0.436433,0.474227,527,0.000000,0.000000,0.000000,40
5,0.009800,0.121099,0.739312,0.736237,0.737771,0.978446,0.592105,0.505618,0.545455,89,0.557464,0.822612,0.664567,513,0.800000,0.636145,0.708725,830,0.000000,0.000000,0.000000,9,0.534602,0.786260,0.636457,393,0.938452,0.877224,0.906806,1686,0.534279,0.428843,0.475789,527,0.000000,0.000000,0.000000,40
6,0.004600,0.125276,0.761704,0.752386,0.757016,0.979420,0.561798,0.561798,0.561798,89,0.602374,0.791423,0.684078,513,0.802053,0.659036,0.723545,830,0.000000,0.000000,0.000000,9,0.601179,0.778626,0.678492,393,0.936914,0.889680,0.912686,1686,0.551867,0.504744,0.527255,527,0.000000,0.000000,0.000000,40


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

[2025-03-05 11:03:30] tartuNLP/EstRoBERTa treenimine lõpetatud
Kokku kulus: 2265.46 sekundit (0.63 tundi)
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.5365853658536586, 'recall': 0.4631578947368421, 'f1': 0.4971751412429378, 'number': 95}
GEP {'precision': 0.6844262295081968, 'recall': 0.7106382978723405, 'f1': 0.6972860125260961, 'number': 470}
LOC {'precision': 0.7062818336162988, 'recall': 0.6656, 'f1': 0.685337726523888, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7357237715803453, 'recall': 0.7578659370725034, 'f1': 0.7466307277628031, 'number': 731}
PER {'precision': 0.9356855739169272, 'recall': 0.9156468531468531, 'f1': 0.9255577645239673, 'number': 2288}
PROD {'precision': 0.5988805970149254, 'recall': 0.5071090047393365, 'f1': 0.5491873396065012, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.8025586353944563
overall_recall 0.7757625721352019
overall_f1 0.7889331377069796
overall_accuracy 0.9809939686919245

Nervaluate tulemused
Strict {'correct': 3764, 'incorrect': 829, 'partial': 0, 

In [8]:
estroberta_combined, estroberta_combined_trainer, estroberta_combined_results = train_model('tartuNLP/EstRoBERTa', 'combined', epochs=6)

COMBINED andmestik laetud


Map:   0%|          | 0/30045 [00:00<?, ? examples/s]

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstRoBERTa and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstRoBERTa EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.725, 'recall': 0.5272727272727272, 'f1': 0.6105263157894736, 'number': 55}
GEP {'precision': 0.7692307692307693, 'recall': 0.8333333333333334, 'f1': 0.8, 'number': 60}
LOC {'precision': 0.4482758620689655, 'recall': 0.6290322580645161, 'f1': 0.523489932885906, 'number': 62}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.5084745762711864, 'recall': 0.5202312138728323, 'f1': 0.5142857142857143, 'number': 173}
PER {'precision': 0.9600456621004566, 'recall': 0.8302073050345509, 'f1': 0.8904182106934887, 'number': 1013}
PROD {'precision': 0.364741641337386, 'recall': 0.3870967741935484, 'f1': 0.37558685446009393, 'number': 310}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.7380050505050505
overall_recall 0.698744769874477
overall_f1 0.7178385016886706
overall_accuracy 0.9609631521342575

Nervaluate tulemused
Strict {'correct': 1169, 'incorrect': 272, 'partial': 0, 'missed': 232, 'spu

Seqeval tulemused
EVE {'precision': 0.46835443037974683, 'recall': 0.3894736842105263, 'f1': 0.4252873563218391, 'number': 95}
GEP {'precision': 0.7134268537074149, 'recall': 0.7574468085106383, 'f1': 0.7347781217750258, 'number': 470}
LOC {'precision': 0.7416520210896309, 'recall': 0.6752, 'f1': 0.7068676716917922, 'number': 625}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8}
ORG {'precision': 0.7636363636363637, 'recall': 0.8043775649794802, 'f1': 0.7834776815456362, 'number': 731}
PER {'precision': 0.9469730446310207, 'recall': 0.9366258741258742, 'f1': 0.9417710393320149, 'number': 2288}
PROD {'precision': 0.6048951048951049, 'recall': 0.5466034755134281, 'f1': 0.5742738589211619, 'number': 633}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.8176470588235294
overall_recall 0.8021434460016488
overall_f1 0.8098210570120682
overall_accuracy 0.9825673534072901

Nervaluate tulemused
Strict {'correct': 3892, 'incorrect': 745, 'partial': 0

In [9]:
estbert_combined, estbert_combined_trainer, estbert_combined_results = train_model('tartuNLP/EstBERT', 'combined', epochs=6)

COMBINED andmestik laetud


Map:   0%|          | 0/30045 [00:00<?, ? examples/s]

Map:   0%|          | 0/3955 [00:00<?, ? examples/s]

Map:   0%|          | 0/4120 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at tartuNLP/EstBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Map:   0%|          | 0/3207 [00:00<?, ? examples/s]

Kombineeritud andmestikul treenitud tartuNLP/EstBERT EWT testandmestikul
Hindan testandmestikul..


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Seqeval tulemused
EVE {'precision': 0.559322033898305, 'recall': 0.5789473684210527, 'f1': 0.5689655172413793, 'number': 57}
GEP {'precision': 0.5797101449275363, 'recall': 0.7272727272727273, 'f1': 0.6451612903225806, 'number': 55}
LOC {'precision': 0.40476190476190477, 'recall': 0.6181818181818182, 'f1': 0.48920863309352525, 'number': 55}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
ORG {'precision': 0.7668711656441718, 'recall': 0.5411255411255411, 'f1': 0.6345177664974619, 'number': 231}
PER {'precision': 0.9613428280773143, 'recall': 0.863013698630137, 'f1': 0.9095283926852743, 'number': 1095}
PROD {'precision': 0.39862542955326463, 'recall': 0.32044198895027626, 'f1': 0.3552833078101072, 'number': 362}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}
overall_precision 0.7798552472858866
overall_recall 0.6970350404312669
overall_f1 0.7361229718189582
overall_accuracy 0.9520137103684662

Nervaluate tulemused
Strict {'correct': 1293, 'incorrect': 237, '

Seqeval tulemused
EVE {'precision': 0.49056603773584906, 'recall': 0.30952380952380953, 'f1': 0.3795620437956205, 'number': 84}
GEP {'precision': 0.7134831460674157, 'recall': 0.6614583333333334, 'f1': 0.6864864864864865, 'number': 384}
LOC {'precision': 0.7045454545454546, 'recall': 0.7732426303854876, 'f1': 0.7372972972972973, 'number': 441}
MUU {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 6}
ORG {'precision': 0.7323162274618585, 'recall': 0.7242798353909465, 'f1': 0.7282758620689657, 'number': 729}
PER {'precision': 0.9217657342657343, 'recall': 0.9063171465406102, 'f1': 0.9139761646803901, 'number': 2327}
PROD {'precision': 0.4930662557781202, 'recall': 0.5423728813559322, 'f1': 0.5165456012913641, 'number': 590}
UNK {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}
overall_precision 0.7849934181658622
overall_recall 0.7841332456717072
overall_f1 0.7845630961517377
overall_accuracy 0.974904596636148

Nervaluate tulemused
Strict {'correct': 3578, 'incorrect': 766, 